# Optimization method : Interior Penalty Function (Section 5.7.2 Haftka)

## For the unconstrained search: conjugated gradient + interval reduction method

1) Step size: Golden Search Method, employing the function "minimize_scalar" from scipy.optimize

2) Search direction : Conjugated Gradient, $\mathbf{d}_{(t)} = -\nabla_{\mathbf{x}} f_{(t)} + \beta_{(t)}\mathbf{d}_{(t-1)}$, onde $\beta_{(t)}=\left[\frac{||\nabla_{\mathbf{x}} f_{(t)}||}{||\nabla_{\mathbf{x}} f_{(t-1)}||}\right]^2$ 

The first step consists in defining the algorithms parameters, such as initial point $\mathbf{x}_{(0)}$, $\alpha_{(t)}$ and convergence tolerance constant $\epsilon_{\nabla}$, as well as the function to be minimized and its gradient evaluation:



In [17]:
import numpy as np
from scipy.optimize import minimize_scalar
# Problem to be solved and variable for computational cost computation
global problem, cost_f, cost_g, r
problem=3
cost_f,cost_g=0,0
# Initial guess
x=np.array([.75, .75])
# Upper bound for the Gold Search algorithm
alpha0=.1
# Initial value for penalization parameter
r=.51
# Convergence Tolerance
TolG=1e-5
# Maximum number of iterations
itmax=4

Define the objective function to be minimized and its constraints (it must be done by the user):

In [18]:
# Definition of the equation to be minimized
def f_obj(x):
    global cost_f, problem
    cost_f=cost_f+1
    if problem==1:
        f = x[0]**2+10*x[1]**2
        df = np.array([2*x[0], 20*x[1]])
    elif problem==2:
        f=(x[0]-1.5)**2+(x[1]-1.5)**2
        df=np.array([2*(x[0]-1.5), 2*(x[1]-1.5)])
    elif problem==3:
        f=(x[0]-1.5)**2+(x[1]-1.5)**2
        df=np.array([2*(x[0]-1.5), 2*(x[1]-1.5)])
    elif problem==4:
        f=(x[0]-1)**2+(x[1]-1)**2
        df=np.array([2*(x[0]-1), 2*(x[1]-1)])
    return f, df

# Definition of the constraints: h and g
def nlconstraints(x):
    global cost_g, problem
    cost_g=cost_g+1
    if problem==1:
        h= x[0]+x[1]-4
        dh = np.array([1, 1])
        g=np.array([])
        dg=np.array([])
    elif problem==2:
        h=x[0]+x[1]-2
        dh=np.array([1, 1])
        g=np.array([])
        dg=np.array(np.zeros(x.shape))
    elif problem==3:
        h=np.array([])
        dh=np.array(np.zeros(x.shape))
        g=np.array([x[0]+x[1]-2])
        dg=np.array([1, 1])
    elif problem==4:
        h=np.array([])
        dh=np.array(np.zeros(x.shape))
        g=np.array([x[0]+x[1]-4, 2-x[0]])
        dg=np.array([[1, 1],[-1,0]])
    return h, dh, g, dg

From here on, the method is user independent:

In [19]:
# Definition of the penalized function phi (only inequality constraints)
def phi(x):
    global r
    f,df=f_obj(x)
    h,dh,g,dg=nlconstraints(x)

    auxg=g
    #ph=f+r*(h.sum()**2 + 1/auxg.sum()**2)
    ph=f-r*(1/auxg.sum())
    # Construction of the gradient of phi: contribuition of the inequality constriants (g_i)
    dgaux=np.array(np.zeros(x.shape))
    if g.size==1:  # splip into two situations: with only one constraints, and more constraints
        dgaux=(-1)*(dg)/(g**2)
    else:
        for i in range(g.size):
            dgaux=dgaux+(-1)*dg[i,:]/(g[i]**2)
        
    # Construction of the gradient of phi: contribuition of the equality constriants (h_j)
    #dhaux=np.array(np.zeros(x.shape))
    #if h.size==1:  # slip into two situations: with only one constraints, and more constraints
    #    dhaux=dh*h
    #else:
    #    for j in range(h.size):
    #        dhaux=dhaux+dh[j,:]*h[j]
        
        
    #dph=df+2*r*(dhaux + dgaux)
    dph=df-r*(dgaux)
    return ph, dph

# Definition of the equation to be minimized as function of the step size alpha
def f_alpha(alpha,args):
    xk,d=args[0],args[1]
    xaux=xk+alpha*d
    f,df=phi(xaux)
    return f

Definition of the unconstrained optimization algorithm:


In [20]:
def CG_GS(x,alpha0,TolG):
    # Count variable
    t=0
    # f and df values at the inital point
    [f,df]=phi(x)
    dftm1=df
    while np.sqrt(df @ df)>TolG:
        # Search direction: Conjugated Gradient
        beta = (np.linalg.norm(df)/np.linalg.norm(dftm1))**2
 
        if t==0:
            d=-df
        else:
            d=-df+beta*dtm1
            
        # Step determination: Golden Search (method='golden'), Brent (method='brent') or Bounded (method='bounded')
        alpha=minimize_scalar(f_alpha, bounds=(.001, alpha0), args=([x,d]), method='bounded')

        # Update the current point 
        xt=x+alpha.x*d
    
        # Saves information of gradient and descent direction of current iteration
        dftm1=df
        dtm1=d
    
        # Evaluate the objective funciton and gradient at the new point
        [f,df]=phi(xt)
    
        # Update the design vairable and iteration number 
        x=xt
        t=t+1
    return x,f,df,t

Interior Penalty (or Barrier) method iterative scheme:

In [21]:
k=0
while k<itmax:
    xt,f,df,t=CG_GS(x,alpha0,TolG)
    r=r/5
    x=xt
    k=k+1
    
fopt,dfopt=f_obj(xt)
hopt,dhopt,gopt,dgopt=nlconstraints(xt)

The optimum design is stored in the variable $x$. Once the results are obtained, we may print them:

In [22]:
fopt,dfopt=f_obj(x)
hopt,dhopt,gopt,dgopt=nlconstraints(x)
print('Optimum found:')
print(x)
print('Objective function value at the optimum:')
print(fopt)
print('Inequality constraints at the optimum:')
print(gopt)
print('Equality constraints at the optimum:')
print(hopt)

print('Number of times that the f_obj function and constraints were evaluated, respectively:')
print(cost_f)
print(cost_g)
print('Number of iterations of the External penalty method:')
print(k)


Optimum found:
[0.96900859 0.96900859]
Objective function value at the optimum:
0.5639037569256715
Inequality constraints at the optimum:
[-0.06198282]
Equality constraints at the optimum:
[]
Number of times that the f_obj function and constraints were evaluated, respectively:
94
94
Number of iterations of the External penalty method:
4


In [ ]:
dg=np.zeros((1,xt.size))

dg[0,0]=1#np.array([1, 1])
dg[0,1]=3
aaa=np.zeros((1,xt.size))
print(dg[0,1])

In [ ]:
from random import randint
aaa=randint(0, 9)
print(aaa)

In [ ]:
print(randint(0,18))